In [0]:
!pip install tsfresh
!pip install catboost

    100% |████████████████████████████████| 1.2MB 6.4MB/s 
    100% |████████████████████████████████| 55.5MB 504kB/s 
  [enum]
You must restart the runtime in order to use newly installed versions.


In [0]:
# The essentials
import pandas as pd
import numpy as np

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt

# Progress bars
from tqdm import tqdm_notebook


# TSFRESH Feature Extraction
from tsfresh import extract_features
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_selection.relevance import calculate_relevance_table

from catboost import CatBoostRegressor

# Access our Google Drive
from google.colab import drive

/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [0]:
drive.mount('/content/drive', force_remount=True)
!ls "/content/drive/My Drive/Rinse Over Run"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
20178.png
20451.png
20899.png
22112.png
22369.png
22414.png
22487.png
23011.png
23142.png
23599.png
23872.png
24804.png
24845.png
24872.png
25129.png
25908.png
25983.png
26270.png
27115.png
27243.png
27346.png
27366.png
27418.png
27508.png
all_train_preds_per_phase.p
baseline_features_with_preds_per_phase.csv
baseline_model_per_nunique_phases.csv
dtw_distances_3.p
extended_phase_predictors.csv
last_cleaned_test.csv
last_cleaned_train.csv
mds_embeddings_2d_3.cs

In [0]:
PROCESS_COMBINATION = 15

phases = {
    15: ['pre_rinse', 'caustic', 'intermediate_rinse', 'acid'],
    3:  ['pre_rinse', 'caustic'],
    7:  ['pre_rinse', 'caustic', 'intermediate_rinse'],
    1:  ['pre_rinse'],
    8:  ['acid'],
    2:  ['caustic'],
    6:  ['caustic', 'intermediate_rinse'],
    14: ['caustic', 'intermediate_rinse', 'acid'],
}

ts_cols = [
    'supply_flow',
    'supply_pressure',
    'return_temperature',
    'return_conductivity',
    'return_turbidity',
    'return_flow',
    'tank_level_pre_rinse',
    'tank_level_caustic',
    'tank_level_acid',
    'tank_level_clean_water',
    'tank_temperature_pre_rinse',
    'tank_temperature_caustic',
    'tank_temperature_acid',
    'tank_concentration_caustic',
    'tank_concentration_acid'
]

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_values.csv', index_col=0, parse_dates=['timestamp'])
train_features = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_features_{}.csv'.format(PROCESS_COMBINATION), index_col='process_id')

process_sample = np.random.choice(list(set(train_df['process_id'])), replace=False, size=250)
filtered_train_df = train_df[(train_df['process_id'].isin(process_sample))]

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
extraction_settings = EfficientFCParameters()
filtered_funcs = ['number_cwt_peaks', 'fft_aggregated']#['number_cwt_peaks', 'cwt_coefficients', 
filtered_settings = {}
for func in filtered_funcs:
  filtered_settings[func] = extraction_settings[func]
  
ts_features = extract_features(filtered_train_df[['process_id', 'timestamp'] + ts_cols], 
                               column_id='process_id', column_sort="timestamp", 
                               column_kind=None, column_value=None,
                               impute_function=impute, 
                               default_fc_parameters=filtered_settings,
                               show_warnings=False)

Feature Extraction:  40%|████      | 2/5 [02:04<03:11, 63.95s/it]Process ForkPoolWorker-3:


KeyboardInterrupt: ignored

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/tsfresh/utilities/distribution.py", line 38, in _function_with_partly_reduce
    results = list(itertools.chain.from_iterable(results))
  File "/usr/local/lib/python3.6/dist-packages/tsfresh/utilities/distribution.py", line 37, in <genexpr>
    results = (map_function(chunk, **kwargs) for chunk in chunk_list)
  File "/usr/local/lib/python3.6/dist-packages/tsfresh/feature_extraction/extraction.py", line 302, in _do_extraction_on_chunk
    return list(_f())
  File "/usr/local/lib/python3.6/dist-packages/tsfresh/feature_extraction/extraction.py", line 296, in _f
    for key, item in resu

In [0]:
labels_df = pd.read_csv('/content/drive/My Drive/Rinse Over Run/train_labels.csv', index_col='process_id')
ts_labels = labels_df.loc[ts_features.index]['final_rinse_total_turbidity_liter']

In [0]:
rel_table = calculate_relevance_table(ts_features, ts_labels, ml_task='regression')

In [0]:
def custom_mape(approxes, targets):
    return np.mean(np.abs(np.subtract(approxes, targets)) / np.maximum(np.abs(targets), 290000))

class MAPEMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, targets, weight):
        return custom_mape(np.exp(approxes), np.exp(targets)), len(targets)

In [0]:
relevant_features = list(rel_table[rel_table['p_value'] < 0.005]['feature'].values)

print(relevant_features)

val_index = list(np.random.choice(list(ts_features.index), replace=False, size=int(0.1*len(ts_features))))
train_index = list(set(ts_features.index) - set(val_index))

X_train = ts_features.loc[train_index, :][relevant_features]
X_val = ts_features.loc[val_index, :][relevant_features]
y_train = np.log(ts_labels.loc[train_index])
y_val = np.log(ts_labels.loc[val_index])

print(X_train.shape)

cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100, learning_rate=0.33,
                        loss_function='MAPE', eval_metric=MAPEMetric())
cat.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=50)

['supply_flow__fft_coefficient__coeff_0__attr_"abs"', 'supply_flow__fft_coefficient__coeff_0__attr_"real"', 'return_flow__fft_coefficient__coeff_0__attr_"abs"', 'return_flow__fft_coefficient__coeff_0__attr_"real"', 'return_flow__fft_coefficient__coeff_4__attr_"abs"', 'supply_flow__fft_coefficient__coeff_18__attr_"abs"', 'supply_flow__fft_coefficient__coeff_20__attr_"abs"', 'supply_flow__fft_coefficient__coeff_12__attr_"abs"', 'supply_flow__fft_coefficient__coeff_19__attr_"abs"', 'return_flow__fft_coefficient__coeff_12__attr_"abs"', 'return_flow__fft_aggregated__aggtype_"skew"', 'supply_flow__fft_coefficient__coeff_29__attr_"abs"', 'supply_flow__fft_coefficient__coeff_27__attr_"abs"', 'return_flow__fft_coefficient__coeff_22__attr_"abs"', 'return_flow__fft_coefficient__coeff_11__attr_"abs"', 'supply_flow__fft_coefficient__coeff_21__attr_"abs"', 'supply_flow__fft_coefficient__coeff_25__attr_"abs"', 'supply_flow__fft_coefficient__coeff_16__attr_"abs"', 'supply_flow__fft_coefficient__coeff_

In [0]:
X_train_orig = train_features.loc[train_index, :]
X_val_orig = train_features.loc[val_index, :]
y_train = np.log(ts_labels.loc[train_index])
y_val = np.log(ts_labels.loc[val_index])

print(X_train.shape)

cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100, learning_rate=0.33,
                        loss_function='MAPE', eval_metric=MAPEMetric())
cat.fit(X_train_orig, y_train, eval_set=(X_val_orig, y_val), verbose=50)

(225, 1560)
0:	learn: 0.8203349	test: 0.7756854	best: 0.7756854 (0)	total: 211ms	remaining: 5h 51m 40s
50:	learn: 0.8203305	test: 0.7756799	best: 0.7756799 (50)	total: 4.6s	remaining: 2h 30m 13s
100:	learn: 0.8203147	test: 0.7756600	best: 0.7756600 (100)	total: 5.17s	remaining: 1h 25m 19s
150:	learn: 0.8202597	test: 0.7755910	best: 0.7755910 (150)	total: 5.75s	remaining: 1h 3m 23s
200:	learn: 0.8200679	test: 0.7753501	best: 0.7753501 (200)	total: 6.32s	remaining: 52m 19s
250:	learn: 0.8193992	test: 0.7745163	best: 0.7745163 (250)	total: 6.88s	remaining: 45m 35s
300:	learn: 0.8170692	test: 0.7716194	best: 0.7716194 (300)	total: 7.45s	remaining: 41m 6s
350:	learn: 0.8089497	test: 0.7615549	best: 0.7615549 (350)	total: 8.02s	remaining: 37m 57s
400:	learn: 0.7811185	test: 0.7267028	best: 0.7267028 (400)	total: 8.63s	remaining: 35m 43s
450:	learn: 0.7044378	test: 0.6356218	best: 0.6356218 (450)	total: 11.5s	remaining: 42m 18s
500:	learn: 0.5589667	test: 0.4540461	best: 0.4540461 (500)	total

In [0]:
X_train_comb = X_train_orig.merge(X_train, left_index=True, right_index=True)
X_val_comb = X_val_orig.merge(X_val, left_index=True, right_index=True)
y_train = np.log(ts_labels.loc[train_index])
y_val = np.log(ts_labels.loc[val_index])

print(X_train_comb.shape)

cat = CatBoostRegressor(iterations=100000, od_type='Iter', od_wait=100, learning_rate=0.33,
                        loss_function='MAPE', eval_metric=MAPEMetric())
cat.fit(X_train_comb, y_train, eval_set=(X_val_comb, y_val), verbose=50)

(225, 1815)
0:	learn: 0.8203349	test: 0.7756854	best: 0.7756854 (0)	total: 1.69s	remaining: 1d 22h 56m 10s
50:	learn: 0.8203305	test: 0.7756797	best: 0.7756797 (50)	total: 41.3s	remaining: 22h 30m 4s
100:	learn: 0.8203145	test: 0.7756596	best: 0.7756596 (100)	total: 45.8s	remaining: 12h 35m 19s
150:	learn: 0.8202590	test: 0.7755898	best: 0.7755898 (150)	total: 50.4s	remaining: 9h 15m 4s
200:	learn: 0.8200656	test: 0.7753470	best: 0.7753470 (200)	total: 55.1s	remaining: 7h 36m 10s
250:	learn: 0.8193929	test: 0.7745076	best: 0.7745076 (250)	total: 59.9s	remaining: 6h 36m 40s
300:	learn: 0.8170548	test: 0.7716146	best: 0.7716146 (300)	total: 1m 5s	remaining: 5h 59m 18s
350:	learn: 0.8089070	test: 0.7615170	best: 0.7615170 (350)	total: 1m 11s	remaining: 5h 40m 29s
400:	learn: 0.7810304	test: 0.7270682	best: 0.7270682 (400)	total: 1m 20s	remaining: 5h 32m 15s
450:	learn: 0.7023391	test: 0.6386845	best: 0.6386845 (450)	total: 2m 3s	remaining: 7h 34m 1s
500:	learn: 0.5588698	test: 0.4683922	b

In [0]:
X_train_orig.merge(X_train, left_index=True, right_index=True)

,pipeline_L1,pipeline_L10,pipeline_L11,pipeline_L12,pipeline_L2,pipeline_L3,pipeline_L4,pipeline_L6,pipeline_L7,pipeline_L8,...,"tank_temperature_caustic__fft_coefficient__coeff_35__attr_""angle""","tank_concentration_acid__fft_coefficient__coeff_98__attr_""abs""","tank_level_clean_water__fft_coefficient__coeff_50__attr_""abs""","return_conductivity__fft_coefficient__coeff_98__attr_""imag""","return_temperature__fft_coefficient__coeff_30__attr_""imag""","return_turbidity__fft_coefficient__coeff_52__attr_""abs""","return_temperature__fft_coefficient__coeff_37__attr_""abs""","tank_concentration_acid__fft_coefficient__coeff_52__attr_""abs""","return_flow__fft_coefficient__coeff_3__attr_""real""","tank_concentration_acid__fft_coefficient__coeff_79__attr_""abs"""
21506,0,0,0,0,0,0,0,0,1,0,...,158.245481,0.503584,14.282342,50.193839,126.044519,179.913664,112.321341,0.284854,-2.788027e+06,0.847825
26114,0,0,0,0,0,0,0,0,0,0,...,67.144189,0.679442,2.515987,-1.411325,104.412540,0.107140,103.080526,2.444253,-8.840396e+05,0.353634
20996,0,0,0,0,0,0,0,0,1,0,...,155.087924,2.331945,40.454337,100.219088,332.076657,124.611070,142.859137,25.010438,-8.392686e+05,14.415431
21508,0,0,0,0,0,0,1,0,0,0,...,43.638345,1.342913,5.936901,43.512613,232.585708,101.062566,354.062537,5.489035,3.223815e+05,1.190057
26635,1,0,0,0,0,0,0,0,0,0,...,-75.255975,0.103598,0.676182,79.941481,347.012149,242.854029,297.455599,0.795607,1.775170e+05,0.320319
21005,0,0,0,0,0,0,0,0,1,0,...,-19.433338,5.177926,3.290378,30.441993,270.966560,81.828021,145.151963,12.437735,-1.045318e+06,7.168053
27661,0,0,0,0,0,0,0,0,0,0,...,-78.104843,0.367323,10.419736,61.967972,189.090150,54.716146,127.437915,1.183818,-1.081704e+06,0.721306
25615,1,0,0,0,0,0,0,0,0,0,...,61.905277,2.696260,9.999627,-11.355278,38.794149,59.033426,143.626899,6.050834,1.414702e+04,3.505053
21010,0,0,0,0,0,0,0,0,0,0,...,15.480379,0.190566,4.437441,48.948838,138.492518,0.797127,146.917931,0.417376,-7.877499e+05,0.181300
22034,0,0,0,0,0,0,0,0,0,0,...,-58.579848,2.070589,14.322537,6.978690,105.975359,166.872344,135.779218,6.191923,-1.115687e+06,3.707463
